In [11]:
import numpy as np
import MDAnalysis as mda

In [12]:
topology =  "am5.0nm2ac1.0nc8953hl12.5sd0.1dt0.0005bdump2000adump5000tdump5000ens1.bug.data"
trajectory = "am5.0nm2ac1.0nc8953hl12.5sd0.1dt0.0005bdump2000adump5000tdump5000ens1.bug.lammpstrj"

cell = mda.Universe(topology, trajectory, topology_format='DATA',
        format='LAMMPSDUMP', lammps_coordinate_convention='unscaled',
        atom_style="id type x y z", dt=0.0005*2000)

In [46]:
bug = cell.select_atoms("type 1")
odims = np.roll(np.arange(3),-2)[1:]
print(odims)
print(bug.positions)
print(bug.positions[:, odims])

[0 1]
[[10.36382  20.66004  11.28848 ]
 [10.37299  20.594189 16.41315 ]]
[[10.36382  20.66004 ]
 [10.37299  20.594189]]


In [42]:
np.arctan2(bug.positions[:, odims[1]], bug.positions[:, odims[0]])

array([1.1058408, 1.1042049], dtype=float32)

In [35]:
sliced_trj = cell.trajectory[0: -1]
type(sliced_trj)

MDAnalysis.coordinates.base.FrameIteratorSliced

In [25]:
import os
import re
from typing import Dict, List, Literal, ClassVar, Optional
from abc import ABC, abstractmethod
from collections import OrderedDict
import numpy as np

def number_density_cube(
    n_atom: float,
    d_atom: float,
    l_cube: float,
    pbc: Optional[bool] = False
) -> float:
    """
    Compute the bulk number density of a species in a cubic box.

    Parameters
    ----------
    n_atom : float
        Number of particles.
    d_atom : float
        Diameter of the particle of the species.
    l_cube : float
        Length of one side of the cubic box.
    pbc : bool, optional
        Periodic boundary conditions along all box sides. If `True`,
        :math:`v_{avail} = l_{cube}^3`; otherwise,
        :math:`v_{avail} = (l_{cube} - d_{atom})^3`. Defaults to `False`.

    Returns
    -------
    float
        Bulk number density of the species in the cubic box.

    Notes
    -----
    The bulk number density is calculated as :math:`n_{atom} / v_{avail}`,
    where `v_{avail}` is the available volume to the center of geometry of a
    particle based on the presence of periodic boundary conditions.
    """
    v_avail = (l_cube - int(pbc) * d_atom) ** 3
    return n_atom / v_avail


def volume_fraction_cube(
    n_atom: float,
    d_atom: float,
    l_cube: float,
    pbc: Optional[bool] = False
) -> float:
    """
    Compute the volume fraction of a species in a cubic box.

    Parameters
    ----------
    n_atom : float
        Number of particles.
    d_atom : float
        Diameter of the particle of the species.
    l_cube : float
        Length of one side of the cubic box.
    pbc : bool, optional
        Periodic boundary conditions along all box sides. If `True`,
        :math:`v_{avail} = l_{cube}^3`; otherwise,
        :math:`v_{avail} = (l_{cube} - d_{atom})^3`. Defaults to `False`.

    Returns
    -------
    float
        Volume fraction of the species in the cubic box.

    Notes
    -----
    The volume fraction is computed in the volume available to the center of
    geometry of each particle. For point-like particles, the notion of volume
    fraction is meaningless. For finite-size particles, the available volume
    depends on whether periodic boundary conditions (PBCs) are applied.
    """
    rho = number_density_cube(n_atom, d_atom, l_cube, pbc)
    return rho * np.pi * d_atom ** 3 / 6


def number_density_cylinder(
    n_atom: float,
    d_atom: float,
    l_cyl: float,
    d_cyl: float,
    pbc: Optional[bool] = False
) -> float:
    """
    Compute the bulk number density of a species in a cylindrical confinement.

    Parameters
    ----------
    n_atom : float
        Number of particles.
    d_atom : float
        Diameter of the particle of the species.
    l_cyl : float
        Length of the cylindrical confinement.
    d_cyl : float
        Diameter of the cylindrical confinement.
    pbc : bool, optional
        Periodic boundary conditions along the longitudinal axis. If `True`,
        :math:`v_{avail} = \\pi * l_{cyl} * (d_{cyl} - d_{atom})^2 / 4`;
        otherwise, :math:`v_{avail} = \\pi * (l_{cyl} - d_{atom}) * (d_{cyl}
        - d_{atom})^2 / 4`. Defaults to `False`.

    Returns
    -------
    float
        Bulk number density of the species in the cylindrical confinement.

    Notes
    -----
    The bulk number density is calculated as :math:`n_{atom} / v_{avail}`,
    where `v_{avail}` is the available volume to the center of geometry of a
    particle based on the presence of periodic boundary conditions.
    """
    v_avail = np.pi * (l_cyl - int(pbc) * d_atom) * (d_cyl - d_atom) ** 2 / 4
    return n_atom / v_avail


def volume_fraction_cylinder(
    n_atom: float,
    d_atom: float,
    l_cyl: float,
    d_cyl: float,
    pbc: Optional[bool] = False
) -> float:
    """
    Compute the volume fraction of a species in a cylindrical confinement.

    Parameters
    ----------
    n_atom : float
        Number of particles.
    d_atom : float
        Diameter of the particle of the species.
    l_cyl : float
        Length of the cylindrical confinement.
    d_cyl : float
        Diameter of the cylindrical confinement.
    pbc : bool, optional
        Periodic boundary conditions along the longitudinal axis. If `True`,
        :math:`v_{avail} = \\pi * l_{cyl} * (d_{cyl} - d_{atom})^2 / 4`;
        otherwise, :math:`v_{avail} = \\pi * (l_cyl - d_{atom}) * (d_{cyl}
        - d_{atom})^2 / 4`. Defaults to `True`.

    Returns
    -------
    float
        Volume fraction of the species in the cylindrical confinement.

    Notes
    -----
    The volume fraction is computed in the volume available to the center of
    geometry of each particle. For point-like particles, the notion of volume
    fraction is meaningless. For finite-size particles, the available volume
    depends on whether periodic boundary conditions (PBCs) are applied.
    """
    rho = number_density_cylinder(n_atom, d_atom, l_cyl, d_cyl, pbc)
    return rho * np.pi * d_atom ** 3 / 6

def invalid_keyword(
    keyword: str,
    valid_keywords: List[str],
    message: Optional[str] = None
) -> None:
    """
    Raises an error if `keyword` is not in `valid_keywords`.

    Parameters
    ----------
    keyword: str
        Name of the `keyword`
    valid_keywords: array of str
        Array of valid keywords
    message: str
        Message to be printed.

    Raises
    ------
    ValueError
        If `keyword` is not in `valid_keywords`.
    """
    if message is None:
        message = " is an invalid option. Please select one of " + \
            f"{valid_keywords} options."
    if keyword not in valid_keywords:
        raise ValueError(f"'{keyword}'" + message)

class ParserBase(ABC):
    """
    Base parser class for extracting information from filenames or file paths
    in a structured project. Designed to enforce lineage, geometry, and group
    conventions across subclasses for specific project types.

    Parameters
    ----------
    artifact : str
        Artifact that is parsed for extracting information. Can be a filename
        or filepath.
    lineage : {'segment', 'whole', 'ensemble_long', 'ensemble', 'space'}
        The lineage of the name, specifying the hierarchical level within the
        project.
    group : str
        The particle group type in the project. Used for specific group-based
        parsing.

    Attributes
    ----------
    filepath : str
        The filepath if `artifact` is a filepath, otherwise "N/A".
    filename : str
        The filename extracted from `artifact` if it is a filepath, otherwise
        `artifact` itself.
    group : {'bug', 'nucleoid', 'all'}
        Particle group type in the project.
    name : str
        The unique name derived from `filename` based on `lineage` and `group`
        conventions.
    project_name : str
        The name of the project class (subclass of `ParserBase`), automatically
        assigned as the subclass's name.
    lineage_genealogy: List[str]
        List of parent lieages for an `artifact` with a given `lineage`.
    attributes: List[str]
        List of attributes specific to an `artifact` with a given `lineage`,
        including parsed and computed attributes.

    Class Attributes
    ----------------
    lineages : list of str
        List of valid lineage types.
    genealogy : dict of lists
        Dictionary defining the hierarchical relationship between lineages.
        Each lineage points to its parent lineages in a hierarchy.

    Abstract Class Properties
    -------------------------
    geometry : str
        Specifies geometry of the system
    topology : str
        Specifies how particles are connected (how) or not in the system.
    groups : List[str]
        Specifies particle groups in the system
    genealogy_attributes : Dict[str, OrderedDict[str, str]]
        Dictionary defining lineage-specific attributes for each lineage type.
    project_attributes : Dict[str, List[str]]
        Dictionary defining project-level attributes that remain constant but
        are not extractable from a filename.

    Abstract Instance Properties
    ----------------------------
    lineage_attributes: List[str]
        List of attributes specific to an `artifact` with a given `lineage`,
        including parsed attributes.
    physical_attributes: List[str]
        List of attributes specific to an `artifact` with a given `lineage`,
        including computed attributes.

    Methods
    -------
    _find_name() -> None
        Parses and sets the unique name based on `lineage` and `group`.
    _set_parents() -> None
        Sets pattern names for each `lineage` based on `_genealogy`.
    _initiate_attributes() -> None
        Defines and initializes subclass-specific attributes. (Abstract method)
    _parse_lineage_name() -> None
        Parses lineage-specific attributes based on the filename.
        (Abstract method)
    _bulk_attributes() -> None
        Computes physical attributes for the current lineage based on primary
        attributes. (Abstract method)
    """
    _lineages: ClassVar[List[str]] = \
        ["segment", "whole", "ensemble_long", "ensemble", "space"]
    _genealogy: ClassVar[Dict[str, List[str]]] = {
        "segment": ["segment", "whole", "ensemble_long", "ensemble", "space"],
        "whole": ["whole", "ensemble_long", "ensemble", "space"],
        "ensemble_long": ["ensemble_long", "ensemble", "space"],
        "ensemble": ["ensemble", "space"],
        "space": ["space"],
    }

    @property
    @abstractmethod
    def _geometry(self) -> str:
        """
        Defines the system geometry for the parser subclass.
        """

    @property
    @abstractmethod
    def _groups(self) -> List[str]:
        """
        List of valid group names for the subclass.
        """

    @property
    @abstractmethod
    def _topology(self) -> str:
        """
        Defines the polymer topology for the parser subclass.
        """

    @property
    @abstractmethod
    def _genealogy_attributes(self) -> Dict[str, OrderedDict[str, str]]:
        """
        Dictionary of lineage-specific attributes. Each key is a lineage type,
        and each value is an OrderedDict mapping attribute names to their
        short-form representations.
        """

    @property
    @abstractmethod
    def _project_attributes(self) -> Dict[str, List[str]]:
        """
        Dictionary of project attributes. Each key is a lineage type,
        and each value is an OrderedDict mapping attribute names to their
        short-form representations.
        """

    def __init__(
        self,
        artifact: str,
        lineage: Literal["segment", "whole", "ensemble_long", "ensemble",
                         "space"],
        group: str
    ) -> None:
        self._filepath, self._filename = (
            ("N/A", artifact) if '/' not in artifact and '\\' not in artifact
            else os.path.split(artifact)
        )
        invalid_keyword(lineage, self._lineages)
        invalid_keyword(group, self._groups)
        self._lineage = lineage
        self._group = group
        self._project_name = self.__class__.__name__
        self._lineage_genealogy = self._genealogy[lineage]
        self._lineage_attributes = \
            list(self._genealogy_attributes[lineage].keys())
        self._physical_attributes = self._project_attributes[lineage]
        self._lineage_genealogy = self._genealogy[lineage]
        self._attributes = \
            self._lineage_attributes + self._physical_attributes
        self._find_name()

    def __str__(self) -> str:
        """
        Provides a formatted summary of the parser instance.
        """
        observation = (
            f"Arifact:\n"
            f"    Name: '{self.filename}',\n"
            f"    Geometry: '{self._geometry}',\n"
            f"    Group: '{self._group}',\n"
            f"    Lineage: '{self._lineage}',\n"
            f"    Topology: '{self._topology}'"
        )
        return observation

    def __repr__(self) -> str:
        return (
            f"Artifact('{self.filename}' in geometry"
            f" '{self._geometry}' from group '{self._group}' with"
            f" lineage '{self._lineage}' and"
            f" topology '{self._topology}')"
        )

    @property
    def filename(self) -> str:
        """
        Returns the filename, either extracted from the path or the name
        itself.
        """
        return self._filename

    @property
    def filepath(self) -> str:
        """
        Returns the full filepath or 'N/A' if not a valid path.
        """
        return self._filepath

    @property
    def lineage(self) -> str:
        """
        Returns the current lineage.
        """
        return self._lineage

    def _find_name(self) -> None:
        """
        Parses and sets the unique `lineage_name` from the filename
        based on the `lineage` and `group`.

        Notes
        -----
        - For 'segment' and 'whole' lineages, names typically end with the
          group keyword or a hyphen.
        - For 'ensemble_long', 'ensemble', and 'space', names are derived
          from the first substring in the filename.
        """
        if self._lineage in ["segment", "whole"]:
            self._name = \
                self.filename.split("." + self._group)[0].split("-")[0]
        else:
            self._name = self._filename.split("-")[0]

    @property
    def name(self) -> str:
        """
        Returns the unique name parsed from the filename.
        """
        return self._name

    @property
    def project_name(self) -> str:
        """
        Returns the project (parser class) name,
        """
        return self._project_name

    @property
    def attributes(self) -> Dict[str, OrderedDict[str, str]]:
        """
        Returns lineage-specific andp project attributes for an artifact.
        """
        return self._attributes[self._lineage]

    @property
    def lineage_genealogy(self) -> List[str]:
        """
        Returns the parents of a given `lineage`.
        """

    @property
    def lineage_attributes(self) -> List[str]:
        """
        Returns lineage-specific attributes for an artifact with a given
        `lineage`.
        """
    
    @property
    def physical_attributes(self) -> List[str]:
        """
        Returns project-level attributes for an artifact with a given
        `lineage`.
        """

    @property
    def genealogy_attributes(self) -> Dict[str, OrderedDict[str, str]]:
        """
        Returns lineage-specific attributes for the all the lineages.
        """
        return self._genealogy_attributes

    @property
    def project_attributes(self) -> Dict[str, List[str]]:
        """
        Returns project-level attributes for the all the lineages.
        """
        return self._project_attributes

    @abstractmethod
    def _initiate_attributes(self) -> None:
        """
        Defines and initiates the project attributes. Lineage attributes are
        set dynamically via `_parse_name` method.
        """

    def _set_parents(self) -> None:
        """
        Sets parent lineage names for each lineage type, following the
        hierarchy defined in `_genealogy`.

        Notes
        -----
        The `self._genealogy` defines the following parent-child hierarchy:

            - space -> ensemble -> ensemble_long -> whole -> segment

        Each lineage on the left has all the lineages on its right.
        """
        for lineage_name in self._lineages:
            lineage_value = "N/A"
            if lineage_name in self._genealogy[self._lineage]:
                lineage_value = ""
                lineage_attr = self._genealogy_attributes[lineage_name]
                for attr_long, attr_short in lineage_attr.items():
                    lineage_value += \
                            f"{attr_short}{getattr(self, attr_long)}"
            setattr(self, lineage_name, lineage_value)

    @abstractmethod
    def _parse_name(self) -> None:
        """
        Parses lineage attributes from the `name` attribute, assigning them
        dynamically as class attributes.

        Notes
        -----
        Lineage attributes are macroscopic physical attributes of the systems.
        They are added to the class dynamically as new class attribute upon
        class instantiation.
        """

    @abstractmethod
    def _dependant_attributes(self) -> None:
        """
        Calculates system attributes based on parsed values.
        """


class TwoMonDep(ParserBase):
    """
    Extracts structured information about an artifact from its name in the
    *TwoMonDep* project, utilizing specific filename patterns.

    Each lineage level has a unique naming pattern used to parse key physical
    and system attributes:

    - `segment`: am#nm#ac#nc#hl#sd#dt#bdump#adump$tdump#ens#.j#
      One of multiple chunks of a complete artifact.
    - `whole`: am#nm#ac#nc#hl#sd#dt#bdump#adump$tdump#ens#
      A complete artifact. It may be a collection of segments.
    - `ensemble_long`: am#nm#ac#nc#hl#sd#dt#bdump#adump$tdump#
      Detailed name for an 'ensemble' artifact.
    - `ensemble`: nm#am#ac#nc#sd#
      Short name for an 'ensemble' artifact.
    - `space`: nm#am#ac#
      A 'space' artifact.

    For the above four lineages, the short names (eywords) are physical
    attributes where their values (shown by "#" sign) are float or integer
    number. See `genealogy_attributes` below for long names of attribues.

    Other than attributes inhertied from the parent class `ParserBase` as
    explained below, this class dynamically defines new attributes based on the
    list of physical attributes of a given `lineage` as define in the
    `genealogy_attributes` class attribute.

    Parameters
    ----------
    artifact : str
        Name to be parsed, either a filename or filepath.
    lineage : {'segment', 'whole', 'ensemble_long', 'ensemble', 'space'}
        Type of the lineage of the name.
    group : {"bug", "all"}
        Particle group type, with `bug` representing a single polymer.

    Attributes
    ----------
    dmon : float
        Size (diameter) of a monomer. Its associated keyword is 'am'.
    nmon : int
        Number of monomers. Its associated keyword is 'N'.
    dcrowd: float
        Size (diameter) of a crowder. Its associated keyword is 'ac'.
    ncrowd : int
        Number of crowders. Its associated keyword is 'nc'.
    lcube : float
        Length of the simulation box, inferred from 'hl' keyword
        (half-length of hthe simulation box).
    d_sur : float
        Surface-to-surface distance between two monomers fixed in space.
        Its associated keyword is 'sd'.
    dt : float
        Simulation timestep. Its associated keyword is 'dt'.
    bdump : int
        Frequency by which 'bug' configurations are dumped in a 'bug'
        trajectory file. Its associated keyword is 'bdump'.
    adump : int
        Frequency by which 'all' configurations are dumped in a 'segment'
        trajectory file. Its associated keyword is 'adump'.
    tdump : int
        Frequency by which 'themo' variables are written in a 'lammps'
        log file. Its associated keyword is 'tdump'.
    ensemble_id : int
        The ensemble number of a 'whole' artifact in an ensemble. Its
        associated keyword is 'ens'.
    segment_id : int
        The 'segment_id' keyword starts with 'j', ends with a 'padded'
        number such as '05' or '14', showing the succession of segments
        in a artifact file. Its associated keyword is 'j'.
    rho_bulk_m : float
        Bulk number density fraction of monomers.
    phi_bulk_m : float
        Bulk volume fraction of monomers
    rho_bulk_c : float
        Bulk number density fraction of crowders
    phi_bulk_c : float
        Bulk volume fraction of crowders
    space : str
        A space's name.
    ensemble : str, "N/A"
        An ensemble's name if applicable, otherwise "N/A"
    ensemble_long : str, "N/A"
        The name of ensemble derived from 'whole' name if applicable,
        otherwise "N/A"
    whole : str, "N/A"
        A whole's name if applicable, otherwise "N/A"
    segment : str, "N/A"
        A segment's name if applicable, otherwise "N/A"
    lineage_attributes: List[str]
        List of attributes specific to an `artifact` with a given `lineage`,
        including parsed attributes.
    physical_attributes: List[str]
        List of attributes specific to an `artifact` with a given `lineage`,
        including computed attributes.
    attributes: List[str]
        List of attributes specific to an `artifact` with a given `lineage`,
        including parsed and computed attributes.
    lineage_genealogy: List[str]
        List of parent lieages for an `artifact` with a given `lineage`.

    Class Attributes
    ----------------
    _geometry : str
        Specifies geometry of the system
    _topology : str
        Specifies how particles are connected (how) or not in the system.
    _groups : List[str]
        Specifies particle groups in the system
    _genealogy_attributes : Dict[str, Dict[str, str]]
        Maps `lineage` names to attribute keywords for parsing.
    _project_attributes : Dict[str, List[Optional[str]]]
        Specifies additional physical attributes for each `lineage`.

    Examples
    --------
    Creating a instance to parse a filename with specified lineage and group.

    >>> artifact = (
    ..."nm2am5.0ac1.0"
    ..."space",
    ..."bug"
    ... )
    >>> print(artifact.nmon)
    2
    """
    _geometry: ClassVar[str] = "cubic"
    _topology: ClassVar[str] = "atomic"
    _groups: ClassVar[List[str]] = ["bug", "all"]

    _genealogy_attributes: ClassVar[Dict[str, OrderedDict[str, str]]] = {
        # Pattern: am#nm#ac#nc#hl#sd#dt#bdump#adump$tdump#ens#.j#
        "segment": OrderedDict({
            "dmon": "am", "nmon": "nm", "dcrowd": "ac", "ncrowd": "nc",
            "lcube": "hl", "d_sur": "sd", "dt": "dt", "bdump": "bdump",
            "adump": "adump", "tdump": "tdump", "ensemble_id": "ens",
            "segment_id": "j"}
            ),
        # Pattern: am#nm#ac#nc#hl#sd#dt#bdump#adump$tdump#ens#
        "whole": OrderedDict({
            "dmon": "am", "nmon": "nm", "dcrowd": "ac", "ncrowd": "nc",
            "lcube": "hl", "d_sur": "sd", "dt": "dt", "bdump": "bdump",
            "adump": "adump", "tdump": "tdump", "ensemble_id": "ens"}
            ),
        # Pattern: am#nm#ac#nc#hl#sd#dt#bdump#adump$tdump# :
        "ensemble_long": OrderedDict({
            "dmon": "am", "nmon": "nm", "dcrowd": "ac", "ncrowd": "nc",
            "lcube": "hl", "d_sur": "sd", "dt": "dt", "bdump": "bdump",
            "adump": "adump", "tdump": "tdump"}
            ),
        # Pattern: nm#am#ac#nc#sd :
        "ensemble": OrderedDict(
            {"dmon": "am", "nmon": "nm", "dcrowd": "ac", "ncrowd": "nc",
             "d_sur": "sd"}
             ),
        # pttern: nm#am#ac# :
        "space": OrderedDict(
            {"dmon": "am", "nmon": "nm", "dcrowd": "ac", "ncrowd": "nc"}
            )
    }

    _project_attributes: ClassVar[Dict[str, List[str]]] = {
        "segment": ["phi_bulk_m", "rho_bulk_m", "phi_bulk_c", "rho_bulk_c"],
        "whole": ["phi_bulk_m", "rho_bulk_m", "phi_bulk_c", "rho_bulk_c"],
        "ensemble_long": ["phi_bulk_m", "rho_bulk_m", "phi_bulk_c",
                          "rho_bulk_c"],
        "ensemble": [],
        "space": []
        }

    def __init__(
        self,
        artifact: str,
        lineage: str,
        group: Literal["bug", "all"]
    ) -> None:
        super().__init__(artifact, lineage, group)
        self._initiate_attributes()
        self._parse_name()
        self._set_parents()
        if self.lineage in ["segment", "whole", "ensemble_long"]:
            self._dependant_attributes()

    def _initiate_attributes(self) -> None:
        """
        Defines and initiates the project attributes.

        Notes
        -----
        The negative initial values are unphysical.
        """
        if self._lineage in ["segment", "whole", "ensemble_long"]:
            self.phi_bulk_m: float = -1
            self.rho_bulk_m: float = -1
            self.phi_bulk_c: float = -1
            self.rho_bulk_c: float = -1

    def _parse_name(self) -> None:
        """
        Parses lineage attributes from the `name` attribute, assigning them
        dynamically as class attributes.

        Notes
        -----
        Lineage attributes are macroscopic physical attributes of the systems.
        They are added to the class dynamically as new class attribute upon
        class instantiation.
        """
        name_strs = re.compile(r"([a-zA-Z\-]+)")
        words = name_strs.split(self._name)
        attrs_float = ["dmon", "lcube", "dcrowd", "dt", "d_sur"]
        for attr, keyword in self._genealogy_attributes[self._lineage].items():
            try:
                val = words[words.index(keyword) + 1]
                setattr(self,
                        attr,
                        float(val) if attr in attrs_float else int(float(val)))
                if keyword == "hl":
                    # Cube full side from its half-side
                    setattr(self, attr, 2 * getattr(self, attr))
            except ValueError:
                print(f"'{keyword}' attribute not found in '{self._name}'")

    def _dependant_attributes(self) -> None:
        """
        Calculates system attributes based on parsed values.
        """
        self.rho_bulk_m = number_density_cube(
            getattr(self, "nmon"),
            getattr(self, "dmon"),
            getattr(self, "lcube")
        )
        self.phi_bulk_m = volume_fraction_cube(
            getattr(self, "nmon"),
            getattr(self, "dmon"),
            getattr(self, "lcube")
        )
        self.rho_bulk_c = number_density_cube(
            getattr(self, "ncrowd"),
            getattr(self, "dcrowd"),
            getattr(self, "lcube")
        )
        self.phi_bulk_c = volume_fraction_cube(
            getattr(self, "ncrowd"),
            getattr(self, "dcrowd"),
            getattr(self, "lcube")
        )

In [26]:
a = TwoMonDep(
    "am5.0nm2ac1.0nc8953hl12.5sd5.0dt0.0005bdump2000adump5000tdump5000ens1",
    "whole",
    "bug"
)
for key, val in a.__dict__.items():
    print(key, val)

_filepath N/A
_filename am5.0nm2ac1.0nc8953hl12.5sd5.0dt0.0005bdump2000adump5000tdump5000ens1
_lineage whole
_group bug
_project_name TwoMonDep
_lineage_genealogy ['whole', 'ensemble_long', 'ensemble', 'space']
_lineage_attributes ['dmon', 'nmon', 'dcrowd', 'ncrowd', 'lcube', 'd_sur', 'dt', 'bdump', 'adump', 'tdump', 'ensemble_id']
_physical_attributes ['phi_bulk_m', 'rho_bulk_m', 'phi_bulk_c', 'rho_bulk_c']
_attributes ['dmon', 'nmon', 'dcrowd', 'ncrowd', 'lcube', 'd_sur', 'dt', 'bdump', 'adump', 'tdump', 'ensemble_id', 'phi_bulk_m', 'rho_bulk_m', 'phi_bulk_c', 'rho_bulk_c']
_name am5.0nm2ac1.0nc8953hl12.5sd5.0dt0.0005bdump2000adump5000tdump5000ens1
phi_bulk_m 0.008377580409572781
rho_bulk_m 0.000128
phi_bulk_c 0.3000179096276204
rho_bulk_c 0.572992
dmon 5.0
nmon 2
dcrowd 1.0
ncrowd 8953
lcube 25.0
d_sur 5.0
dt 0.0005
bdump 2000
adump 5000
tdump 5000
ensemble_id 1
segment N/A
whole am5.0nm2ac1.0nc8953hl25.0sd5.0dt0.0005bdump2000adump5000tdump5000ens1
ensemble_long am5.0nm2ac1.0nc8953h

In [47]:
import pandas as pd

# Sample DataFrames with an odd and even number of rows
data_odd = {'bin_center': [-2, -1, 0, 1, 2]}
data_even = {'bin_center': [-2, -1, -1, 1, 2, 3]}

ens_avg_even = pd.DataFrame(data_even)
ens_avg_odd = pd.DataFrame(data_odd)


# Finding the midpoints and extracting ens_pos for each case
df_len_even = len(ens_avg_even)
df_len_odd = len(ens_avg_odd)
#print(df_len_odd)

print(df_len_even % 2 == 0)
print(df_len_odd % 2 == 0)

mid_point_even = df_len_even // 2
print("even mid point")
print(mid_point_even)
mid_point_odd = df_len_odd // 2
print("odd mid point")
print(mid_point_odd)

# - (df_len_even % 2 == 0)
ens_neg_even = ens_avg_even.iloc[:mid_point_even].copy()
ens_pos_even = ens_avg_even.iloc[mid_point_even+(df_len_even % 2 == 1):].copy()

ens_neg_odd = ens_avg_odd.iloc[:mid_point_odd].copy()
ens_pos_odd = ens_avg_odd.iloc[mid_point_odd+(df_len_odd % 2 == 1):].copy() 

print("Original DataFrame (even):")
print(ens_avg_even)
print("\nExtracted ens_neg (even):")
print(ens_neg_even)
print("\nExtracted ens_pos (even):")
print(ens_pos_even)


True
False
even mid point
3
odd mid point
2
Original DataFrame (even):
   bin_center
0          -2
1          -1
2          -1
3           1
4           2
5           3

Extracted ens_neg (even):
   bin_center
0          -2
1          -1
2          -1

Extracted ens_pos (even):
   bin_center
3           1
4           2
5           3


In [48]:

print("\nOriginal DataFrame (odd):")
print(ens_avg_odd)
print("\nExtracted ens_neg (odd):")
print(ens_neg_odd)
print("\nExtracted ens_pos (odd):")
print(ens_pos_odd)


Original DataFrame (odd):
   bin_center
0          -2
1          -1
2           0
3           1
4           2

Extracted ens_neg (odd):
   bin_center
0          -2
1          -1

Extracted ens_pos (odd):
   bin_center
3           1
4           2


In [25]:
9 // 2

4

In [11]:
import re

re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$)', 'camelCase')

['camel', 'Case']

In [6]:
test = {'a', 'b', 'c', 'd'}
test_b = set({'a', 'b', 'c', 'd', 'e', 'f'})
test_b.difference(test)

{'e', 'f'}

In [73]:
import numpy as np

lmin = -5
lmax = 5
delta = 0.9
bin_edges = np.arange(lmin, lmax + delta, delta)
bin_center =  0.5 *(bin_edges[:-1] + bin_edges[1:])
print(bin_center)
len(bin_center)

[-4.55 -3.65 -2.75 -1.85 -0.95 -0.05  0.85  1.75  2.65  3.55  4.45  5.35]


12